In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from allfunc import *

In [2]:
def errcube(z,mini_cube,mini_wave):
    true_err = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            spectrum = mini_cube[:,i,j]
            k = 1+z
            select_cont = (mini_wave > 5070*k) & (mini_wave<5090*k)
            select_em = (mini_wave>4997*k) & (mini_wave<5017*k)
       
            signal = np.mean(spectrum[select_cont])
            noise = np.std(spectrum[selet_cont])
            SN = np.sqrt(spectrum/signal)*(signal/noise)
            new_err = spectrum/SN
            true_err[:,i,j] = new_err 
    return true_err

In [3]:
def algorithm_script(obj,z,prefix_path_cube="/media/rickeythecat/Seagate/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))
    k = 1+z
    try:
        (orig_cube,orig_err,orig_wave,orig_header) = loadCube('%s/MUSE/%s/%s.binned.fits'%(prefix_path_cube,obj,obj))
    except IOError:
        (orig_cube,orig_err,orig_wave,orig_header) = loadCube('%s/MUSE/%s/%s.unbinned.fits'%(prefix_path_cube,obj,obj))   
    (cont_cube,cont_err,cont_wave,cont_header) = loadCube('%s/MUSE/%s/fitting/full/%s.cont_model.fits'%(prefix_path_cube,obj,obj))

    difference = difference_in_wavelength_dimension(orig_cube,cont_cube)
    (wo_cube,wo_err,wo_wave,wo_header) = create_wo_absorption_cube(obj,orig_cube,orig_err,orig_header,cont_cube,cont_wave,difference)
    muse_sampling_size = arb_sampling_size(wo_cube)  

    (amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2,m,c) = full_par_central(obj)
    select = (wo_wave>4750*k) & (wo_wave<5090*k)
    mini_wave = wo_wave[select]
    mini_cube = wo_cube[select]
    mini_err = wo_err[select]
    [x0,y0] = agn_location(obj)
    [brightest_pixel_x,brightest_pixel_y] = [int(x0),int(y0)]
    #print brightest_pixel_x,brightest_pixel_y
    data = mini_cube[:,brightest_pixel_y - 50:brightest_pixel_y + 51,brightest_pixel_x - 50:brightest_pixel_x + 51]
    error = mini_err[:,brightest_pixel_y - 50:brightest_pixel_y + 51,brightest_pixel_x - 50:brightest_pixel_x + 51]
      
    popt_mod = [amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2,m,c]
    blr1 = Hb_Fe_doublet_gauss(mini_wave,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1)
    blr2 = Hb_Fe_doublet_gauss(mini_wave,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2)
    #nlr_core = Hb_O3_gauss(mini_wave,amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII)
    nlr_wing = Hb_O3_gauss(mini_wave,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br)
     
    (amp_Hb_blr,x0_Hb_blr,y0_Hb_blr,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr) = ellip_moffat_par(obj)
    (amp_OIII_br,x0_OIII_br,y0_OIII_br,amp_OIII_nr,x0_OIII_nr,y0_OIII_nr) = moffat_O3_par(obj)
    print (x0_OIII_br,y0_OIII_br)
    (x0_OIII_br,y0_OIII_br) = (x0_OIII_br + 28,y0_OIII_br+28)
    (x0_Hb_blr,y0_Hb_blr) = (x0_Hb_blr+28,y0_Hb_blr+28)
    #(amp_OIII_br,x0_OIII_br,y0_OIII_br,amp_OIII_nr,x0_OIII_nr,y0_OIII_nr) = moffat_O3_par(obj)
    psf_BLR = model_eline(amp_Hb_blr,x0_Hb_blr,y0_Hb_blr,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr)
    psf_wing = model_eline(amp_OIII_br,x0_OIII_br,y0_OIII_br,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr)

    noiseless_BLR = (psf_BLR[np.newaxis,:,:]*blr1[:,np.newaxis,np.newaxis]) 
    noiseless_wing = (psf_wing[np.newaxis,:,:]*nlr_wing[:,np.newaxis,np.newaxis]) 
    point_cube = noiseless_BLR + noiseless_wing
    eline_cube = data - point_cube
    eline_err = errcube(z,eline_cube,mini_wave)
    store_cube('%s/%s/%s.point_cube.fits'%(destination_path_cube,obj,obj),point_cube,mini_wave,mini_err,wo_header)
    store_cube('%s/%s/%s.extended_cube.fits'%(destination_path_cube,obj,obj),eline_cube,mini_wave,eline_err,wo_header)

In [4]:
z = {"HE0040-1105":0.041692}

#z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
#    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
#    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
#    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
#    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
#    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
#    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

for obj in objs:
     algorithm_script(obj,z[obj]) 

HE0040-1105


FileNotFoundError: [Errno 2] No such file or directory: '/media/rickeythecat/Seagate/ftp.hidrive.strato.com/users/login-carsftp/IFU_data/MUSE/HE0040-1105/HE0040-1105.unbinned.fits'

In [3]:
z = {"HE1248-1356":0.01465}
objs = z.keys()

for obj in objs:
     algorithm_script(obj,z[obj]) 

HE1248-1356


In [ ]:
psf_wing = model_eline(amp_OIII_br,x0_OIII_br,y0_OIII_br,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr)
    psf_core = model_eline(amp_OIII_nr,x0_OIII_nr,y0_OIII_nr,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr)
    psf_blr = model_eline(amp_Hb_blr,x0_Hb_blr,y0_Hb_blr,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr)
    noiseless_blr = (psf_blr[np.newaxis,:,:]*blr1[:,np.newaxis,np.newaxis]) 
    noiseless_core = (psf_core[np.newaxis,:,:]*nlr_core[:,np.newaxis,np.newaxis]) 
    noiseless_wing = (psf_wing[np.newaxis,:,:]*nlr_wing[:,np.newaxis,np.newaxis]) 
